<a href="https://colab.research.google.com/github/weathon/slay/blob/main/SlayTree_vs_OldCMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import pylab
import numpy as np

In [108]:
import numpy as np
import pandas as pd 
import random
import pylab 
import multiprocessing
import time
from pyrsistent import l
import timeit

def ifNone(a):
    if type(a) == type(None):
        return True
    else:
        return False

def maxPerArg(list1, list2):
    if len(list1) != len(list2):
        raise Exception("OOBA %s,%s" % (len(list1), len(list2)))
    ans = list1[:]
    for i in range(len(list1)):
        if list2[i]>list1[i]:
            ans[i] = list2[i]
    return ans

def minPerArg(list1, list2):
    if len(list1) != len(list2):
        raise Exception("OOBA %s,%s" % (len(list1), len(list2)))
    ans = list1[:]
    for i in range(len(list1)):
        if list2[i]<list1[i]:
            ans[i] = list2[i]
    return ans

class dataPoint:
    def __init__(self, d, id):
        self.data = d
        self.id = id
        self.ancestorsDistances = {}
        self.ancestorsDistancesList = []
        self.distanceFromDad = -1
        self.max = -1
        self.min = -1
        self.fars = [-1]
        self.nears = [-1]
    def __str__(self):
        return str(self.data)

def randomArray():
    size = 10
    ans = []
    for i in range(size):
        ans.append(round(random.random(),5))
    return np.array(ans)



class BallTreeNode:
  def __init__(self):
    self.myobject = None
    self.inner = None
    self.outer = None
    self.left = None
    self.right = None
    self.count = 0

def getData(myArray):
    myArray.sort(key=lambda x: x.distanceFromDad)
    theMax = myArray[-1].distanceFromDad
    theMin = myArray[0].distanceFromDad
    inner = myArray[int(len(myArray)/2)-1].distanceFromDad
    outer = myArray[int(len(myArray)/2)].distanceFromDad
    return theMax, theMin, inner, outer, myArray


counts = 0
def d(a, b, r=0):
  global counts
  a, b = a.data, b.data
  counts+=1
  return np.sqrt(np.sum((a-b)**2))


def ConstructBallTree(BT, localobjects):
  if len(localobjects)==0:
    return BT
  if len(localobjects)==1:
    BT.myobject = localobjects[0]
    BT.inner = 0
    BT.outer = 0
    BT.count = 1
    return BT
  selectedIndex = random.randint(0,len(localobjects)-1)
  localobjects[-1], localobjects[selectedIndex] = localobjects[selectedIndex], localobjects[-1] 
  BT.myobject = localobjects[-1]
  localobjects = localobjects[:-1]
  
  
  for i in range(len(localobjects)):
    localobjects[i].distanceFromDad=d(BT.myobject, localobjects[i])
    localobjects[i].ancestorsDistancesList.append(localobjects[i].distanceFromDad)
  
  BT.myobject.max, BT.myobject.min, BT.inner, BT.outer, sortedlocalobjects = getData(localobjects)

  
  

  L = []
  R = []
  for i in range(len(localobjects)):
    if localobjects[i].distanceFromDad<=BT.inner:
      L.append(localobjects[i])
    elif localobjects[i].distanceFromDad>=BT.outer:
      R.append(localobjects[i])
    else:
      print("where am i", BT.inner,BT.outer)

  BT.count = len(localobjects)

  if len(L) != 0:
    BT.left = ConstructBallTree(BallTreeNode(),L)
  if len(R) != 0:
    BT.right = ConstructBallTree(BallTreeNode(),R)
  
  

  return BT


def generateCMTData(BT):
    if ifNone(BT):
        return [], []
    Lnear, Lfar = generateCMTData(BT.left)
    Rnear, Rfar = generateCMTData(BT.right)
    Lnear, Lfar, Rnear, Rfar = Lnear[:-1], Lfar[:-1], Rnear[:-1], Rfar[:-1]
    
    if len(Lnear) != 0 and len(Rnear) != 0:
        
        BT.myobject.nears = minPerArg(minPerArg(Lnear, Rnear), BT.myobject.ancestorsDistancesList)
        BT.myobject.fars = maxPerArg(maxPerArg(Lfar, Rfar), BT.myobject.ancestorsDistancesList)
    elif len(Lnear) != 0:
        
        BT.myobject.nears = minPerArg(Lnear, BT.myobject.ancestorsDistancesList)
        BT.myobject.fars = maxPerArg(Lfar, BT.myobject.ancestorsDistancesList)
    elif len(Rnear) != 0:
        
        BT.myobject.nears = minPerArg(Rnear, BT.myobject.ancestorsDistancesList)
        BT.myobject.fars = maxPerArg(Rfar, BT.myobject.ancestorsDistancesList)
    elif len(Rnear) == 0 and len(Lnear) == 0:
    
        
        BT.myobject.nears = BT.myobject.ancestorsDistancesList
        BT.myobject.fars = BT.myobject.ancestorsDistancesList 
        
        
        
    
    
    returnNear = BT.myobject.nears
    returnFar = BT.myobject.fars

    return returnNear, returnFar
    



def maxPD(stack, node): 
    nears = node.myobject.nears
    fars = node.myobject.fars
    if len(stack) != len(fars):
        
        raise Exception("stack and fars not match %s, %s" %(len(stack), len(fars)))
    maxPDV = 0
    for i in range(len(stack)):
        
        if fars[i] < stack[i]:
            
            maxPDV = max(maxPDV, stack[i] - fars[i]) 
        elif nears[i] > stack[i]:
            maxPDV = max(maxPDV, nears[i] - stack[i])
    # print(maxPDV)
    return maxPDV
        

def CMTSearchRadius(BT, Object, r, ans):
  global stack
  if BT == None:
    return

  maxPDV = maxPD(stack, BT) 
      
  if maxPDV > r:
    return
    
  dis = d(Object, BT.myobject, r)
    
  if dis <= r:
    ans.append(BT.myobject)  

  # if type(BT.myobject.max) != type(None):
  #   if dis > BT.myobject.max:
  #     if dis - BT.myobject.max > r: 
  #       return

  # if type(BT.myobject.min) != type(None):
  #   if dis < BT.myobject.min:
  #     if BT.myobject.min - dis > r:
  #       return  
  
  stack.append(dis)
  if dis + r >= BT.outer:
    CMTSearchRadius(BT.right, Object, r, ans)
  if BT.inner >= dis - r:                              
    CMTSearchRadius(BT.left, Object, r, ans)
  stack.pop()

In [109]:
class TreeNode:
  def __init__(self, objectA, objectB):
    self.objectA = objectA
    self.objectB = objectB
    if not objectB is None and not objectA is None:
      self.halfway = d(objectA, objectB)
    self.weightOfA = 1
    self.left = None
    self.right = None
    self.ratio = 0.5
  
  def __str__(self):
    return str((self.objectA, self.objectB))

In [110]:
def buildTree(objects):
  if len(objects)==0:
    return None
  if len(objects) == 1:
    root = TreeNode(objects[0], None)
    return root
  elif len(objects) == 2:
    root = TreeNode(objects[0], objects[1])
    return root


  center1 = 0 #why default 50??
  center2 = -1

  maxD = -1

  for i in range(len(objects)):
    dis = d(objects[i], objects[center1])
    if dis>maxD:
      maxD=dis
      center2=i
    
  maxD = -1

  for i in range(len(objects)):
    dis = d(objects[i], objects[center2])
    if dis>maxD:
      maxD=dis
      center1=i
  # print(center1, center2, len(objects))
  root = TreeNode(objects[center1], objects[center2])

  del objects[center1]
  if center1<center2:
     del objects[center2-1] #opmization needed
  else:
    del objects[center2] 
  # I got it center1 and 2 are not indexes!!! mix index and ID again
  left = []
  right = []

  # for _ in range(3):
  #   for i in objects:
  #     if d(i, root.objectA) < d(i, root.objectB):
  #       left.append(i)
  #     else:
  #       right.append(i)
  #   if min(len(left), len(right))/max(len(left), len(right))<=0.8:
  #     if len(left)>len(right):
  #       root.weightOfA *= max(len(left), len(right))/(min(len(left), len(right))+1)*0.01
  #     else:
  #       root.weightOfA *= min(len(left), len(right))/max(len(left), len(right))*0.01
  #   else:
  #     break
  # for i in objects:
  #   if d(i, root.objectA) < d(i, root.objectB):
  #     left.append(i)
  #   else:
  #     right.append(i)
  r = 0.5
  # if len(objects)>=17:
  #   for _ in range(10):
  #     left = []
  #     right = []
  #     for i in objects:
  #       if (1-r)*d(i, root.objectA) < r*d(i, root.objectB):
  #         left.append(i)
  #       else:
  #         right.append(i)
  #     # print(len(left),len(right),min(len(left), len(right))/(len(left)+len(right)),len(objects),r)
      
  #     if 0.44 < min(len(left), len(right))/(len(left)+len(right)):
  #       break
  #     if len(left)>len(right):
  #       r*=0.85
  #     elif len(right)>len(left):
  #       r*=1.15
  #   root.ratio = r
  # else:
  if 1:
    left = []
    right = []
    for i in objects:
      if (1-r)*d(i, root.objectA) < r*d(i, root.objectB):
        left.append(i)
      else:
        right.append(i)
    root.ratio = 0.5
  # print(len(left),len(right))
  # if root.objectA in left:
  # # if root.objectA in left: wtf not in objects but in left? oh i got it it is range objects i got it
  #   print("WTF")
  root.left = buildTree(left)
  root.right = buildTree(right)
  return root

In [111]:
def searchBF(objects, target, r):
  ans = []
  for i in objects:
    if d(target, i)<=r:
      ans.append(i)
  return ans

In [112]:
def searchTree(root, id, r, ans):
  if root == None: #empty, since it is not inited
    return
  if root.objectB == None:
    dA = d(root.objectA, id)
    if dA<=r: #error after using weight, DO INEED TI RESHAPE?
      ans.append(root.objectA)
    return

  dA = d(root.objectA, id)
  dB = d(root.objectB, id)
  if dA <= r:
    ans.append(root.objectA)
  if dB <= r:
    ans.append(root.objectB)

  # halfway = d(root.objectA, root.objectB) #(could be precomputed)
  
  if dA-r<=dB+r: 
  # if (1-root.ratio)*(dA)-r<=root.ratio*(dB)+r: 
    searchTree(root.left,id,r, ans)
  # if root.ratio*(dB)-r<=(1-root.ratio)*(dA)+r: 
  if dB-r<=dA+r: 
    searchTree(root.right,id,r, ans)

In [113]:
# objectSets = []
# for i in range(10000):
#   tmp = dataPoint([random.random(), random.random()], i)

objectSets = []
for i in range(100):
  tmp = dataPoint(randomArray(), i)
  objectSets.append(tmp)


counts = 0

slaytree = buildTree(objectSets)
CMT = ConstructBallTree(BallTreeNode(), objectSets)
generateCMTData(CMT)

([], [])

In [114]:
def BaselineSearchRadius(BT, Object, r, ans):
  if BT == None:
    return
  dis = d(Object, BT.myobject, r)
  if dis <= r:
    ans.append(BT.myobject)
  

  if type(BT.myobject.max) != type(None):
    if dis > BT.myobject.max:
      if dis - BT.myobject.max > r: 
        return

  if type(BT.myobject.min) != type(None):
    if dis < BT.myobject.min:
      if BT.myobject.min - dis > r:
        return  
  
  
  
  if dis + r >= BT.outer:
    BaselineSearchRadius(BT.right, Object, r, ans)
  if BT.inner >= dis - r:                              
    BaselineSearchRadius(BT.left, Object, r, ans)


In [ ]:
import random
for _ in range(1000):

  objectSets_ = []
  totalSize_ = random.randint(10, 2999)

  for i in range(totalSize_):
    tmp = dataPoint(randomArray(), i)
    objectSets_.append(tmp)

  counts = 0

  slaytree = buildTree(objectSets_)

  qID = random.randint(10,totalSize_-1)
  r = round(random.random()*3,2)
  ans = []
  ans2 = []
  ans2 = searchBF(objectSets_, objectSets_[qID], r)
  searchTree(slaytree, objectSets_[qID], r, ans) #have more objects, repeated?
  # ans = list(set(ans))
  usedIDs = []
  # for i in ans: #2 more items in tree, why? always 2
  #   if i.id in usedIDs:
  #     print("Repeated")
  #   usedIDs.append(i.id)
  if len(ans) == len(ans2) == len(set(ans).intersection(ans2)):
    print("WORKING!")
    pass
  else:
    print("Not doing well", qID, r, totalSize_, len(ans), len(ans2), len(set(ans).intersection(ans2))) #did not notice, this is find all items? kkduzikun bf did not get all tree got all
    for i in ans:
      if not d(objectSets_[qID], i)<=r:
        print(i) #kouke zhnganotdoingwel yuoumeiyou cuode shuomiing BF youwentijiaosaunk

Not doing well 641 1.9 648 647 646 646
WORKING!
Not doing well 769 2.17 2014 2014 2012 2012
Not doing well 1553 1.91 1568 1565 1563 1563
Not doing well 203 2.25 462 462 460 460
Not doing well 136 1.87 555 555 553 553
Not doing well 302 2.53 2518 2518 2516 2516
WORKING!
Not doing well 50 2.44 1612 1612 1610 1610
WORKING!
Not doing well 1090 2.61 1808 1808 1806 1806
WORKING!
WORKING!
WORKING!
Not doing well 18 1.19 1844 672 671 671
WORKING!
Not doing well 411 2.28 635 635 633 633
Not doing well 312 1.54 391 348 347 347
Not doing well 232 1.9 490 490 488 488
WORKING!
WORKING!
WORKING!
WORKING!
WORKING!
Not doing well 657 2.15 1475 1475 1473 1473
WORKING!
Not doing well 298 1.95 1197 1197 1195 1195
Not doing well 100 1.56 2714 2485 2484 2484


In [ ]:
a = []
b = []
target = 6
for r in np.arange(0,1,0.01):
  ans1 = searchBF(objectSets, objectSets[target], r)
  ans2 = []
  counts = 0
  searchTree(slaytree, objectSets[target], r, ans2)
  ans3 = []
  stack =[]
  counts1 = counts
  counts = 0
  BaselineSearchRadius(CMT, objectSets[target], r, ans3) #even baseline is better
  counts2 = counts
  print(len(ans1), len(ans2), len(ans3))
  # print(counts1, counts2) #so nervous kunyunexkunyun
  a.append(counts1)
  b.append(counts2)
pylab.plot(a, label="Slay")
pylab.plot(b, label="Trad")
pylab.legend()

In [ ]:
def countHeight(root):
  if root is None:
    return 1
  return max(countHeight(root.left),countHeight(root.right))+1

In [ ]:
countHeight(slaytree)
# log2(10_000)=13.2877124

In [ ]:
def checkBalence(root):
  if root is None:
    return 1
  left = checkBalence(root.left)
  right = checkBalence(root.right)
  print(left, right)
  return left + right + 2
checkBalence(slaytree)